<a href="https://colab.research.google.com/github/samarthya04/AI_Lab/blob/main/lab8_2205498.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assignment 8**: Navigation with Multiple Goals

Objective: Solve a problem where multiple goals exist using search algorithms.

Problem Statement: A robot in a grid needs to collect items (goals) before reachinganexit. Each goal has a different priority or cost.

Tasks:
- Use BFS/DFS for simpler scenarios (unweighted goals).
- Implement A* or Uniform Cost Search for weighted scenarios.
- Analyze the trade-offs between path length and goal priority.

In [1]:
import heapq
from collections import deque

In [2]:
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

In [4]:
def bfs(grid, start, goals, exit):
    queue = deque([(start, 0, [])])
    visited = set()
    while queue:
        (x, y), cost, path = queue.popleft()
        if (x, y) in visited:
            continue
        visited.add((x, y))
        path = path + [(x, y)]
        if (x, y) in goals:
            goals.remove((x, y))
        if not goals and (x, y) == exit:
            return path
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and grid[nx][ny] != '#':
                queue.append(((nx, ny), cost + 1, path))
    return []

In [5]:
def dfs(grid, start, goals, exit):
    stack = [(start, 0, [])]
    visited = set()
    while stack:
        (x, y), cost, path = stack.pop()
        if (x, y) in visited:
            continue
        visited.add((x, y))
        path = path + [(x, y)]
        if (x, y) in goals:
            goals.remove((x, y))
        if not goals and (x, y) == exit:
            return path
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and grid[nx][ny] != '#':
                stack.append(((nx, ny), cost + 1, path))
    return []

In [6]:
def ucs(grid, start, goals, exit, goal_weights):
    pq = [(0, start, [])]
    visited = set()
    while pq:
        cost, (x, y), path = heapq.heappop(pq)
        if (x, y) in visited:
            continue
        visited.add((x, y))
        path = path + [(x, y)]
        if (x, y) in goals:
            cost += goal_weights.get((x, y), 0)
            goals.remove((x, y))
        if not goals and (x, y) == exit:
            return path
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and grid[nx][ny] != '#':
                heapq.heappush(pq, (cost + 1, (nx, ny), path))
    return []

In [7]:
def a_star(grid, start, goals, exit, goal_weights):
    pq = [(0, start, [])]
    visited = set()
    while pq:
        cost, (x, y), path = heapq.heappop(pq)
        if (x, y) in visited:
            continue
        visited.add((x, y))
        path = path + [(x, y)]
        if (x, y) in goals:
            cost += goal_weights.get((x, y), 0)
            goals.remove((x, y))
        if not goals and (x, y) == exit:
            return path
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and grid[nx][ny] != '#':
                new_cost = cost + 1 + heuristic((nx, ny), exit)
                heapq.heappush(pq, (new_cost, (nx, ny), path))
    return []

In [8]:
grid = [
    ["S", 0, 0, 0, "G1"],
    ["#", "#", 0, "#", 0],
    [0, 0, 0, 0, "G2"],
    ["#", 0, "#", "#", 0],
    [0, 0, 0, "E", 0]
]

start = (0, 0)
goals = {(0, 4), (2, 4)}
exit = (4, 3)
goal_weights = {(0, 4): 3, (2, 4): 5}

print("BFS Path:", bfs(grid, start, set(goals), exit))
print("DFS Path:", dfs(grid, start, set(goals), exit))
print("UCS Path:", ucs(grid, start, set(goals), exit, goal_weights))
print("A* Path:", a_star(grid, start, set(goals), exit, goal_weights))

BFS Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (3, 1), (4, 1), (4, 2), (4, 3)]
DFS Path: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (2, 3), (2, 2), (2, 1), (3, 1), (4, 1), (4, 2), (4, 3)]
UCS Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (3, 1), (4, 1), (4, 2), (4, 3)]
A* Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (3, 1), (4, 1), (4, 2), (4, 3)]


**Assignment 9**: AI Planner Using A for Task Scheduling*

Objective: Use A* Search to optimize task scheduling.

Problem Statement: A set of tasks with dependencies and durations needs to be scheduled to minimize total time.

Tasks:
- Represent tasks and dependencies as a directed graph.
- Use A* Search when the heuristic estimates the remaining tasks' duration.
- Compare results with a greedy algorithm.

In [9]:
import heapq

In [10]:
def heuristic(remaining_tasks, durations):
    return sum(durations[t] for t in remaining_tasks)

In [11]:
def a_star_scheduler(tasks, dependencies, durations):
    pq = [(0, [], set(tasks))]
    best_time = float('inf')
    best_schedule = []

    while pq:
        cost, schedule, remaining_tasks = heapq.heappop(pq)

        if not remaining_tasks:
            if cost < best_time:
                best_time = cost
                best_schedule = schedule
            continue

        available_tasks = [t for t in remaining_tasks if all(dep in schedule for dep in dependencies.get(t, []))]

        for task in available_tasks:
            new_schedule = schedule + [task]
            new_remaining = remaining_tasks - {task}
            new_cost = cost + durations[task]
            est_cost = new_cost + heuristic(new_remaining, durations)
            heapq.heappush(pq, (est_cost, new_schedule, new_remaining))

    return best_schedule, best_time

In [12]:
def greedy_scheduler(tasks, dependencies, durations):
    schedule = []
    remaining_tasks = set(tasks)
    total_time = 0

    while remaining_tasks:
        available_tasks = [t for t in remaining_tasks if all(dep in schedule for dep in dependencies.get(t, []))]
        if not available_tasks:
            return [], float('inf')
        next_task = min(available_tasks, key=lambda t: durations[t])
        schedule.append(next_task)
        remaining_tasks.remove(next_task)
        total_time += durations[next_task]

    return schedule, total_time

In [13]:
tasks = {"A", "B", "C", "D", "E"}
dependencies = {"B": ["A"], "C": ["A"], "D": ["B", "C"], "E": ["D"]}
durations = {"A": 3, "B": 2, "C": 1, "D": 4, "E": 2}

print("A* Schedule:", a_star_scheduler(tasks, dependencies, durations))
print("Greedy Schedule:", greedy_scheduler(tasks, dependencies, durations))

A* Schedule: (['A', 'B', 'C', 'D', 'E'], 36)
Greedy Schedule: (['A', 'C', 'B', 'D', 'E'], 12)
